# Average of Frobenius traces 

The notebook contains code that calculates average of $a_p(E_D)$ values as $D$ increases. Interesting thing to note is that there is no murmuration-like pattern. However, these averages tend to zero for every $p$. 

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import os
from sympy import primerange
import nbformat



parquet_file = "frobenius_cn1k.parquet"

In [2]:
# === STEP 2: Load data ===
df = pd.read_parquet(parquet_file, engine="pyarrow")

# === STEP 3: Setup ===
num_primes = 1000
trace_cols = [f'a{i}' for i in range(1, num_primes + 1)]
if not all(col in df.columns for col in trace_cols) or 'n' not in df.columns:
    raise ValueError("Missing 'n' or one or more a1 to a1000 columns.")

df.loc[:, 'mod8'] = df['n'] % 8

In [10]:
# === Generate first 1000 primes for tooltips ===
primes = list(primerange(0, 7920))[:num_primes]  # 7920 > 1000th prime

# === STEP 4: Plot ===
fig = go.Figure()
colors = ['teal', 'red', 'green', 'blue', 'orange', 'purple', 'brown', 'black']

#for residue in range(8):
#    group = df[df['mod8'] == residue]
#    if group.empty:
#        continue

sqrt_p = np.sqrt(primes)
noramalized = df[trace_cols].values / (2 * sqrt_p)
naverages = noramalized.mean(axis=0)

averages = df[trace_cols].mean().values
indices = np.arange(1, num_primes + 1)

fig.add_trace(go.Scatter(
   x=indices,
   y=naverages,
   mode='markers',
   name=f'n ≡ {residue} mod 8',
   marker=dict(size=6, color=colors[residue]),
   hovertemplate=(
            'Index: %{x}<br>'
            'Prime: %{customdata[0]}<br>'
            'Average aₚ: %{y:.3f}<extra></extra>'
   ),
   customdata=np.array([[p] for p in primes])
))

# === STEP 5: Layout ===
fig.update_layout(
    title='Average Frobenius Traces (Hover Shows Prime)',
    xaxis_title='Prime index',
    yaxis_title='Average trace',
    template='plotly_white',
    hovermode='closest',
    legend_title='Residue mod 8'
)

fig.show()
fig.write_html("frobenius_plot_normal.html")

In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from scipy.interpolate import make_interp_spline
from sympy import primerange
import os

# === CONFIG ===
output_dir = "plotsf"
os.makedirs(output_dir, exist_ok=True)
use_normalization = False      # Set to False to plot raw a_p
spline_smooth = True          # Set to False to skip smoothed plot


# Primes and sqrt(p)

num_traces = num_primes
sqrt_p = np.sqrt(primes)
indices = np.arange(1, num_traces + 1)

# Color palette for 8 classes
colors = ['black', 'red', 'green', 'blue', 'orange', 'purple', 'brown', 'teal']

# === SCATTER PLOT OF ALL CLASSES ===
scatter_fig = go.Figure()

for residue in range(8):
    group = df[df['mod8'] == residue]
    if group.empty:
        continue

    data_matrix = group[trace_cols].values
    if use_normalization:
        data_matrix = data_matrix / (2 * sqrt_p)  # Normalization

    averages = data_matrix.mean(axis=0)

    scatter_fig.add_trace(go.Scatter(
        x=indices,
        y=averages,
        mode='markers',
        name=f'n ≡ {residue} mod 8',
        marker=dict(size=6, color=colors[residue]),
        customdata=np.array([[p] for p in primes]),
        hovertemplate='Index: %{x}<br>Prime: %{customdata[0]}<br>aₚ: %{y:.3f}<extra></extra>'
    ))

# Add Sato–Tate bounds (if normalized)
if use_normalization:
    scatter_fig.add_trace(go.Scatter(
        x=indices,
        y=np.ones_like(indices),
        mode='lines',
        name='Sato-Tate Bound (+1)',
        line=dict(dash='dash', color='gray')
    ))
    scatter_fig.add_trace(go.Scatter(
        x=indices,
        y=-np.ones_like(indices),
        mode='lines',
        name='Sato-Tate Bound (−1)',
        line=dict(dash='dash', color='gray')
    ))

scatter_fig.update_layout(
    title='Average of Frobenius traces across residues mod 8 (scatter plot)',
    xaxis_title='Prime Index',
    yaxis_title='Average of normalized a_p values' if use_normalization else 'Average of a_p values',
    template='plotly_white',
    hovermode='x unified',
    legend_title='Residue mod 8'
)

scatter_fig.write_html(f"{output_dir}/all_residues_scatter_raw.html")
#scatter_fig.write_image(f"{output_dir}/traces_avg_scatter_normal.jpeg", engine="orca")

# === SMOOTHED SPLINE PLOT ===
if spline_smooth:
    spline_fig = go.Figure()

    for residue in range(8):
        group = df[df['mod8'] == residue]
        if group.empty:
            continue

        data_matrix = group[trace_cols].values
        if use_normalization:
            data_matrix = data_matrix / (2 * sqrt_p)

        averages = data_matrix.mean(axis=0)

        # Cubic spline interpolation
        spline = make_interp_spline(indices, averages, k=3)
        x_smooth = np.linspace(indices.min(), indices.max(), 1000)
        y_smooth = spline(x_smooth)

        spline_fig.add_trace(go.Scatter(
            x=x_smooth,
            y=y_smooth,
            mode='lines',
            name=f'n ≡ {residue} mod 8',
            line=dict(color=colors[residue])
        ))

    if use_normalization:
        spline_fig.add_trace(go.Scatter(
            x=x_smooth,
            y=np.ones_like(x_smooth),
            mode='lines',
            name='Sato-Tate Bound (+1)',
            line=dict(dash='dash', color='gray')
        ))
        spline_fig.add_trace(go.Scatter(
            x=x_smooth,
            y=-np.ones_like(x_smooth),
            mode='lines',
            name='Sato-Tate Bound (−1)',
            line=dict(dash='dash', color='gray')
        ))

    spline_fig.update_layout(
        title='Smoothed Frobenius traces by mod 8 (Spline)',
        xaxis_title='Prime Index',
        yaxis_title='Average of normalized a_p values' if use_normalization else 'Average of a_p values',
        template='plotly_white',
        hovermode='x unified',
        legend_title='Residue mod 8'
    )

    spline_fig.write_html(f"{output_dir}/all_residues_spline_raw.html")
    #spline_fig.write_image(f"{output_dir}/traces_avg_smooth_normal.jpeg", engine="orca")